In [1]:
import math

import cv2
import numpy
import numpy as np
from matplotlib import pyplot

# Ustawienie rozmarów wyświetlanych obrazów
pyplot.rcParams["figure.figsize"] = (18, 10)

In [2]:
#
# Wczytanie obrazów źródłowych
#
# źródło grafiki colors.jpg: https://unsplash.com/photos/gT5kuls6Y6Q
# źródło grafiki gacław-na-kuchni.jpg: własne
#
#image_from_file = cv2.imread('images/colors.jpg')
image_from_file = cv2.imread('./images/kitty.jpg')
image_gray = cv2.cvtColor(image_from_file, cv2.COLOR_BGR2GRAY)
image_color = cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB)
print('Rozmiar obrazka: ', image_from_file.shape)

Rozmiar obrazka:  (768, 1024, 3)


# Zadanie 1

In [3]:
output = numpy.copy(image_gray)


#
# Algorytm
#

def clamp(val, min_val=0, max_val=255) -> float:
    return min(max(val, min_val), max_val)


def find_closest_pixel(val, k):
    return clamp(round((k - 1) * val / 255) * 255 / (k - 1))


def floyd_steinberg_single_channel(grey_img, k=3) -> None:
    for y in range(grey_img.shape[0]):
        for x in range(grey_img.shape[1]):
            old_pixel = grey_img[y][x]
            new_pixel = find_closest_pixel(old_pixel, k)
            grey_img[y][x] = np.uint8(new_pixel)
            quent_error = old_pixel - new_pixel
            if x + 1 < grey_img.shape[1]:
                grey_img[y][x + 1] = np.uint8(grey_img[y][x + 1] + quent_error * (7 / 16))
            if y + 1 < grey_img.shape[0]:
                grey_img[y + 1][x] = np.uint8(grey_img[y + 1][x] + quent_error * (5 / 16))
                if x - 1 > 0:
                    grey_img[y + 1][x - 1] = np.uint8(grey_img[y + 1][x - 1] + quent_error * (3 / 16))
                if x + 1 < grey_img.shape[1]:
                    grey_img[y + 1][x + 1] = np.uint8(grey_img[y + 1][x + 1] + quent_error / 16)


#
# Wyświetlenie
#
floyd_steinberg_single_channel(output)
pyplot.imshow(output, cmap='gray')

KeyboardInterrupt: 

In [ ]:
#
# Histogram
#
histr = cv2.calcHist([output], [0], None, [256], [0, 256])
pyplot.plot(histr)
pyplot.xlim([-1, 256])
pyplot.xlabel('Wartośc składowej koloru []')
pyplot.ylabel('Liczba pikseli obrazu []')

# Zadanie 2

In [ ]:
output = numpy.copy(image_color)


#
# Algorytm
#

def floyd_steinberg(transform_image, k=3):
    floyd_steinberg_single_channel(transform_image[:, :, 0], k)
    floyd_steinberg_single_channel(transform_image[:, :, 1], k)
    floyd_steinberg_single_channel(transform_image[:, :, 2], k)


#
# Wyświetlenie
#
floyd_steinberg(output)
pyplot.imshow(output)

In [ ]:
#
# Histogram
#
color = ('r', 'g', 'b')

for i, col in enumerate(color):
    histr = cv2.calcHist([output], [i], None, [256], [0, 256])
    pyplot.plot(histr, color=col)
    pyplot.xlim([-1, 256])
    pyplot.xlabel('Wartośc składowej koloru []')
    pyplot.ylabel('Liczba pikseli obrazu []')

# Zadanie 3

In [ ]:
#
# Przygotowanie płótna
#
width = 80
height = 60
image = numpy.zeros((height, width, 3), dtype=numpy.uint8)


#
# Funkcja rysująca punkt
#
# NOTE(sdatko): punkt 0,0 to lewy dolny róg obrazu
#
def draw_point(image, x, y, color=(255, 255, 255)):
    image[image.shape[0] - 1 - y, x, :] = color


#
# Funkcja rysująca linię
#
def draw_line(image, x1, y1, x2, y2):
    delta_x, delta_y = int(math.fabs(x2 - x1)), int(math.fabs(y2 - y1))
    x_i, y_i = np.sign(x2 - x1), np.sign(y2 - y1)
    if delta_x > delta_y:
        d = 2 * delta_y - delta_x
    else:
        d = 2 * delta_x - delta_y
    current_x, current_y = x1, y1
    draw_point(image, current_x, current_y)

    while current_x != x2 or current_y != y2:
        if delta_x > delta_y:
            current_x += x_i
            d += 2 * delta_y
        if delta_y > delta_x:
            current_y += y_i
            d += 2 * delta_x
        if d >= 0:
            if delta_x > delta_y:
                current_y += y_i
                d -= 2 * delta_x
            if delta_y > delta_x:
                current_x += x_i
                d -= 2 * delta_y
        draw_point(image, current_x, current_y)


#
# Funkcja rysująca trójkąt
#
def area_of_triangle(a, b, c):
    return (c[0] - a[0]) * (b[1] - a[1]) - (c[1] - a[1]) * (b[0] - a[0])


def is_point_in_triangle(p, a, b, c):
    a_1 = area_of_triangle(a, b, p)
    a_2 = area_of_triangle(b, c, p)
    a_3 = area_of_triangle(c, a, p)

    if a_1 == 0 or a_2 == 0 or a_3 == 0:
        return True

    sign = np.sign(a_1) + np.sign(a_2) + np.sign(a_3)
    return math.fabs(sign) == 3


def draw_triangle(image, a, b, c):
    x_min = min(a[0], b[0], c[0])
    x_max = max(a[0], b[0], c[0])
    y_min = min(a[1], b[1], c[1])
    y_max = max(a[1], b[1], c[1])
    
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            if x_min <= x <= x_max and y_min <= y <= y_max and is_point_in_triangle((x, y), a, b, c):
                draw_point(image, x, y)


#
# Rysowanie
#
draw_line(image, 0, 0, 20, 20)
draw_triangle(image, (0, 30), (25, 0), (50, 50))
#
# Wyświetlenie
#
pyplot.imshow(image)

# Zadanie 4

In [4]:
#
# Przygotowanie płótna
#
width = 80
height = 60
image = numpy.zeros((height, width, 3), dtype=numpy.uint8)


#
# Funkcja rysująca punkt
#
# NOTE(sdatko): punkt 0,0 to lewy dolny róg obrazu
#
def draw_point(image, x, y, color=(255, 255, 255)):
    image[image.shape[0] - 1 - y, x, :] = color


#
# Funkcja rysująca linię
#
def draw_line(image, x1, y1, x2, y2, c1, c2):
    delta_x, delta_y = int(math.fabs(x2 - x1)), int(math.fabs(y2 - y1))
    x_i, y_i = np.sign(x2 - x1), np.sign(y2 - y1)
    
    points = []
    
    if delta_x > delta_y:
        d = 2 * delta_y - delta_x
    else:
        d = 2 * delta_x - delta_y
    
    current_x, current_y = x1, y1
    points.append((current_x, current_y))
    while current_x != x2 or current_y != y2:
        if delta_x > delta_y:
            current_x += x_i
            d += 2 * delta_y
        if delta_y > delta_x:
            current_y += y_i
            d += 2 * delta_x
        if d >= 0:
            if delta_x > delta_y:
                current_y += y_i
                d -= 2 * delta_x
            if delta_y > delta_x:
                current_x += x_i
                d -= 2 * delta_y
        points.append((current_x, current_y))
    t_space = np.linspace(0, 1, len(points))
    colors = [c1 + t * (c2 - c1) for t in t_space]
    # for point, color in (points, colors):
    #     draw_point(image, point[0], point[1], color)

#
# Funkcja rysująca trójkąt
#
def draw_triangle(image, a, b, c):
    pass  # TODO: implement


#
# Rysowanie
#

draw_line(image, 0, 0, 50, 50, np.array([255, 0, 0]), np.array([0, 0, 255]))

#
# Wyświetlenie
#
pyplot.imshow(image)

KeyboardInterrupt: 

# Zadanie 5

In [ ]:
#
# Przygotowanie płótna
#
width = 80
height = 60
image = numpy.zeros((height, width, 3), dtype=numpy.uint8)


#
# Funkcja rysująca punkt
#
# NOTE(sdatko): punkt 0,0 to lewy dolny róg obrazu
#
def draw_point(image, x, y, color=(255, 255, 255)):
    image[image.shape[0] - 1 - y, x, :] = color


#
# Funkcja rysująca linię
#
def draw_line(image, x1, y1, x2, y2):
    pass  # TODO: implement


#
# Funkcja rysująca trójkąt
#
def draw_triangle(image, a, b, c):
    pass  # TODO: implement


#
# Rysowanie
#
pass  # TODO: implement

#
# Wyświetlenie
#
pyplot.imshow(image)